In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('../data/Semi Finals Case/train_data.csv')
promo_df = pd.read_csv('../data/Semi Finals Case/promotion_data.csv')
val_df = pd.read_csv('../data/Semi Finals Case/validation_data.csv')
test_df = pd.read_csv('../data/Semi Finals Case/test_data.csv')

In [3]:
train_df.head()

,CategoryCode,ItemCode,DateID,DailySales
0,category_2,1071115,12/19/2021,1
1,category_4,210652,2/13/2022,1
2,category_2,877624,12/19/2021,1
3,category_1,371104,12/19/2021,3
4,category_1,1090294,12/19/2021,3


In [4]:
promo_df.head()

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,DiscountType,SellingPrice
0,379249,12/3/2021,12/9/2021,10,Percentage,70
1,1066570,11/19/2021,11/25/2021,15,Percentage,45
2,124954,10/29/2021,11/4/2021,15,Percentage,210
3,35530,10/22/2021,10/28/2021,15,Percentage,60
4,1013335,1/21/2022,1/27/2022,10,Percentage,210


In [7]:
print(promo_df.shape, promo_df['ItemCode'].nunique())

(314, 6) 66


In [10]:
train_df['DateID'] = pd.to_datetime(train_df['DateID'])
promo_df['PromotionStartDate'] = pd.to_datetime(promo_df['PromotionStartDate'])
promo_df['PromotionEndDate'] = pd.to_datetime(promo_df['PromotionEndDate'])

In [ ]:
prms_dict = promo_df.groupby('ItemCode')['PromotionStartDate'].agg(list).to_dict()
prme_dict = promo_df.groupby('ItemCode')['PromotionEndDate'].agg(list).to_dict()
prmp_dict = {item: list(zip(prms_dict[item], prme_dict[item])) for item in prme_dict}
prmp_dict

In [29]:
sub_dfs = []
for item in promo_df['ItemCode'].values:
    flt_df = train_df.copy().loc[train_df['ItemCode']==item, :]
    for ps, pe in prmp_dict[item]:
        sub_df = flt_df.copy().loc[(flt_df['DateID']>ps)&(flt_df['DateID']<pe), :]
        promo_row = list(promo_df.copy().loc[(promo_df['ItemCode']==item)&(promo_df['PromotionStartDate']==ps), :].values[0])
        sub_df['PromoStarted'] = promo_row[1]
        sub_df['PromoEnded'] = promo_row[2]
        sub_df['DiscountValue'] = promo_row[3]
        sub_df['DiscountType'] = promo_row[4]
        sub_df['SellingPrice'] = promo_row[5]
        sub_dfs.append(sub_df)
    

In [28]:
train_df.shape

(20651, 4)

In [30]:
full_flt_df = pd.concat(sub_dfs)

In [31]:
full_flt_df.shape

(8912, 9)

In [32]:
full_flt_df.head()

,CategoryCode,ItemCode,DateID,DailySales,PromoStarted,PromoEnded,DiscountValue,DiscountType,SellingPrice
2059,category_1,379249,2021-12-04,9,2021-12-03,2021-12-09,10,Percentage,70
2483,category_1,379249,2021-12-07,11,2021-12-03,2021-12-09,10,Percentage,70
9468,category_1,379249,2021-12-05,9,2021-12-03,2021-12-09,10,Percentage,70
9636,category_1,379249,2021-12-08,9,2021-12-03,2021-12-09,10,Percentage,70
19537,category_1,379249,2021-12-06,6,2021-12-03,2021-12-09,10,Percentage,70


In [33]:
full_flt_df.to_csv('../data/Semi Finals Case/merged_promo_item_df.csv', index=False)